<a href="https://colab.research.google.com/github/hayleypc/HawaiiClimate/blob/main/HI_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
!pip install diptest
import diptest

from google.colab import drive

from shapely.geometry import Point

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc
from sklearn.model_selection import cross_val_score

from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.7/195.7 kB 15.3 MB/s eta 0:00:00


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# Functions

In [67]:
# Preprocess data function
def preprocess_data(matched_data,scaler = None):
    matched_data['distance'] = 0
    matched_data = matched_data[[str(i) == '20' for i in matched_data["depth_adj_bottom"]] ]
    matched_data['imp_c_float'] = matched_data['imp_c'].astype(float)

    # Select ID fields and numeric columns
    id_fields = matched_data[['source_dataset', 'island', 'dist_id', 'soil_column_id', 'unique_id', 'depth_top',
                              'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude', 'x_sample', 'y_sample',
                              'x_driver', 'y_driver']]
    keep_cols = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                 'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI', 'aet', 'def',
                 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs',
                 'agbd_m', 'agbd_sd', 'agbd_n']

    numeric_cols = matched_data[keep_cols].replace('', np.nan).astype(float).fillna(0)

    # # Scale numeric columns
    if scaler is None:
      scaler = MinMaxScaler()
      scaler = scaler.fit(numeric_cols)

    scaled_numeric_cols = scaler.transform(numeric_cols)
    scaled_numeric_df = pd.DataFrame(scaled_numeric_cols, columns=numeric_cols.columns, index=numeric_cols.index)

    # # Scale imp_c
    min_c = matched_data['imp_c_float'].min()
    max_c = matched_data['imp_c_float'].max()
    scaled_imp_c = (matched_data['imp_c_float'] - min_c) / (max_c - min_c)

    # Combine ID fields with scaled numeric data
    numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)
    numeric_df['imp_c_scaled'] = scaled_imp_c

    return numeric_df, scaler, min_c, max_c
    # return numeric_df


In [12]:
# Train model function
def train_model(preprocessed_data):
    keep_cols = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                 'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI', 'aet', 'def',
                 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs',
                 'agbd_m', 'agbd_sd', 'agbd_n']
    X = preprocessed_data[keep_cols]
    y = preprocessed_data['imp_c_scaled']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Build and compile model
    model = Sequential([
        Dense(256, activation='relu', input_dim=X_train.shape[1]),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(512, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    # Train model
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=1)

    test_loss = model.evaluate(X_test, y_test)
    predictions = model.predict(X_test).flatten()
    r_squared = r2_score(y_test, predictions)

    return model, test_loss, r_squared, predictions, y_test

In [13]:
# Predict on reserve function
def predict_on_reserve(preprocessed_data, model, min_c, max_c):
    keep_cols = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI', 'aet', 'def',
                'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs',
                'agbd_m', 'agbd_sd', 'agbd_n']
    X = preprocessed_data[keep_cols]
    predictions = model.predict(X)

    inversed_predictions = predictions * (max_c - min_c) + min_c
    inversed_truth = preprocessed_data['imp_c_scaled'] * (max_c - min_c) + min_c

    df_out = preprocessed_data.copy()
    df_out['predictions'] = predictions
    df_out['inversed_predictions'] = inversed_predictions
    df_out['inversed_imp_c'] = inversed_truth

    return df_out

In [14]:
# Evaluate model function
def evaluate_model(model, X_test, y_test, inversed_predictions, inversed_truth):
    test_loss = model.evaluate(X_test, y_test)
    r_squared = r2_score(inversed_truth, inversed_predictions)

    print("Test Loss:", test_loss)
    print("R-Squared Score:", r_squared)

    mae = mean_absolute_error(inversed_truth, inversed_predictions)
    rmse = mean_squared_error(inversed_truth, inversed_predictions, squared=False)

    print("Mean Absolute Error (MAE):", mae)
    print("Root Mean Squared Error (RMSE):", rmse)

    # Scatter plot of true vs predicted values
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=inversed_truth, y=inversed_predictions)
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title("True vs Predicted Values")
    plt.plot([min(inversed_truth), max(inversed_truth)], [min(inversed_truth), max(inversed_truth)], 'r')
    plt.show()

    # Residual plot
    residuals = inversed_truth - inversed_predictions
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True)
    plt.xlabel("Residuals")
    plt.title("Distribution of Residuals")
    plt.show()


In [15]:
# Cross-validation and model training
def build_model(input_shape, output_shape):
    model = Sequential([
        Dense(256, activation='relu', input_dim=input_shape),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(512, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(output_shape, activation='linear')
    ])
    return model

In [8]:
# Define function for rescaling
def rescale_to_minus_one_one(array):
    return 2 * (array - array.min()) / (array.max() - array.min()) - 1


In [9]:
def train_model_a(preprocess_data):

    keep_cols = ['water', 'trees','grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI','aet', 'def',
                'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn','tmmx', 'vap', 'vpd', 'vs',
                'agbd_m', 'agbd_sd', 'agbd_n']

    X = preprocess_data[keep_cols]

    y = preprocess_data['imp_c_scaled']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define model
    def build_model(input_shape):
        model = Sequential([
            Dense(256, activation='relu', input_dim=input_shape),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(512, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1, activation='linear')
        ])
        return model

    model = build_model(X_train.shape[1])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=0)

    test_loss = model.evaluate(X_test, y_test)
    predictions = model.predict(X_test).flatten()
    r_squared = r2_score(y_test, predictions)

    return model, test_loss, r_squared, predictions, y_test

# Load and preprocess data

In [16]:
# Load data
gdf = gpd.read_file('/content/drive/MyDrive/hawaii_soils/HI soils data/2024_update/annotated_combo_imputed_SOC.gpkg')
drivers_gpd = gpd.read_file('/content/drive/MyDrive/hawaii_soils/Analysis Data/250_summary_grid_dt.gpkg')
soils_csv = pd.read_csv('/content/drive/MyDrive/hawaii_soils/HI soils data/combined_soc_2024_04_05.csv')
labeled_dist = pd.read_csv('/content/drive/MyDrive/hawaii_soils/labeled_distr_annote.csv')

In [30]:
# Preprocess soils data
soils_csv = soils_csv.dropna(subset=['latitude', 'longitude'])
soils_csv['geometry'] = soils_csv.apply(lambda row: Point(float(row['longitude']), float(row['latitude'])), axis=1)
soils_gpd = gpd.GeoDataFrame(soils_csv, geometry='geometry', crs="EPSG:4326")

# Merge and transform GeoDataFrames
soils_gpd = pd.merge(soils_gpd, gdf[['dist_id', 'unique_id']], on='unique_id', how='inner')
soils_gpd = soils_gpd.to_crs(drivers_gpd.crs)

# Perform spatial join
matched_data = gpd.sjoin_nearest(soils_gpd, drivers_gpd, how='left', distance_col='distance')

/usr/local/lib/python3.10/dist-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [31]:
# Additional preprocessing steps
drivers_gpd['x_driver'] = drivers_gpd.geometry.x
drivers_gpd['y_driver'] = drivers_gpd.geometry.y
soils_gpd['x_sample'] = soils_gpd.geometry.x
soils_gpd['y_sample'] = soils_gpd.geometry.y

In [32]:
soils_gpd = pd.merge(soils_gpd, labeled_dist,  on='dist_id')

In [33]:
# Reproject both GeoDataFrames to the same CRS
soils_buffered = soils_gpd.to_crs(epsg=32604)
drivers_gpd = drivers_gpd.to_crs(soils_buffered.crs)

# Apply buffer in the same CRS
soils_buffered['x_sample'] = soils_buffered.geometry.x
soils_buffered['y_sample'] = soils_buffered.geometry.y
soils_buffered.geometry = soils_buffered.geometry.buffer(1000)

# Perform spatial join
matches_within_distance = gpd.sjoin(soils_buffered, drivers_gpd, how='left', predicate='intersects')

In [56]:
matches_within_distance = pd.merge(labeled_dist, matches_within_distance, on='dist_id')

In [38]:
keep_cols = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
              'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI', 'aet', 'def',
              'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs',
              'agbd_m', 'agbd_sd', 'agbd_n']

numeric_cols = drivers_gpd[keep_cols].replace('', np.nan).astype(float).fillna(0)

In [39]:
global_scaler = MinMaxScaler()
global_scaler = global_scaler.fit(numeric_cols)

In [68]:
# Preprocess data and get scaled values
numeric_df, scaler, min_c, max_c = preprocess_data(matches_within_distance,scaler=global_scaler)

<ipython-input-67-515e653087b1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_data['imp_c_float'] = matched_data['imp_c'].astype(float)


In [70]:
global_scaler.transform(drivers_gpd[keep_cols])

array([[0.01971653, 0.12002313, 0.70418756, ..., 0.00233966, 0.00482294,
        0.25806452],
       [0.02109351, 0.31880874, 0.51522931, ..., 0.00957597, 0.02872613,
        0.22580645],
       [0.02058384, 0.37253208, 0.41890304, ..., 0.0052422 , 0.01274947,
        0.27419355],
       ...,
       [0.01593273, 0.5023984 , 0.12296759, ..., 0.0135864 , 0.01395604,
        0.22580645],
       [0.01965143, 0.48245845, 0.1508686 , ..., 0.01201557, 0.02169049,
        0.25806452],
       [0.02497371, 0.34661654, 0.20406703, ..., 0.01391951, 0.02132026,
        0.40322581]])

In [71]:
# Assuming numeric_df needs to have dist_id included, merge or join it if necessary
# numeric_df = numeric_df.merge(matches_within_distance[['dist_id']], on='some_common_column', how='left')

dict_list = []

for dist_id in matches_within_distance['dist_id'].unique():
    dist_df = pd.DataFrame({
        "dist_id": [dist_id],
        "labeled_dist": soils_gpd[(soils_gpd['dist_id'] == dist_id) & (soils_gpd['depth_adj_bottom'] == 20)]['labeled_dist_h'].unique(),
        "imp_c": [soils_gpd[(soils_gpd['dist_id'] == dist_id) & (soils_gpd['depth_adj_bottom'] == 20)]['imp_c'].tolist()],
        "imp_c_quantile": [np.quantile(soils_gpd[(soils_gpd['dist_id'] == dist_id) & (soils_gpd['depth_adj_bottom'] == 20)]['imp_c'], np.linspace(0, 1, 10)).tolist()]
    })
    dict_list.append(dist_df)

# Concatenate all DataFrames in dict_list to form a final DataFrame if needed
dist_df = pd.concat(dict_list, ignore_index=True)


# Prepare random forest classifier

## Random forest specific functions

In [72]:
## Bimodal and multimodal generators

def generate_bimodal_distribution(size, mean1, std1, mean2, std2, proportion1):
    """
    Generate a random bimodal distribution.

    Parameters:
        size (int): Total number of samples to generate.
        mean1 (float): Mean of the first distribution.
        std1 (float): Standard deviation of the first distribution.
        mean2 (float): Mean of the second distribution.
        std2 (float): Standard deviation of the second distribution.
        proportion1 (float): Proportion of samples from the first distribution.

    Returns:
        np.array: An array of samples from the bimodal distribution.
    """
    size1 = int(size * proportion1)
    size2 = size - size1
    samples1 = np.random.normal(mean1, abs(std1), size1)
    samples2 = np.random.normal(mean2, abs(std2), size2)
    return np.concatenate([samples1, samples2])

def multimodal_generator(n, mean, sd, mean_variation, sd_variation):
    # bimodal_data = generate_bimodal_distribution(size=1000, mean1=-1*mean, std1=np.random.normal(0, sd_variation, 1), mean2=mean, std2=np.random.normal(0, sd_variation, 1), proportion1=np.random.uniform(.4, .6, 1)[0])
    # bi_dist =  np.quantile(bimodal_data, np.linspace(0, 1, n))
    variable_mean = mean + np.random.normal(0, mean_variation)
    variable_sd = sd + abs(np.random.normal(0, sd_variation))
    dist = np.random.normal(variable_mean, variable_sd, int(n*1.5))
    dist.sort()
    dist_a = dist[-int(n):]

    variable_mean = mean+(1+mean*1.5+sd*1.5) + np.random.normal(0, mean_variation)
    variable_sd = sd + abs(np.random.normal(0, sd_variation))
    dist = np.random.normal(variable_mean, variable_sd, int(n*1.5)) +(1+mean_variation*1.5+sd*1.5)
    dist.sort()
    dist_b = dist[:int(n)]

    bi_dist = np.concatenate([dist_a,dist_b])
    return bi_dist

In [73]:
def normal_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation)
    variable_sd = sd + abs(np.random.normal(0, sd_variation))
    return np.random.normal(variable_mean, variable_sd, n)

def uniform_generator(n, min_val, max_val, min_variation, max_variation):
    variable_min = min_val + np.random.uniform(-min_variation, min_variation)
    variable_max = max_val + np.random.uniform(-max_variation, max_variation)
    return np.random.uniform(variable_min, variable_max, n)

def left_tailed_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation)
    variable_sd = sd + abs(np.random.normal(0, sd_variation))
    dist = np.random.normal(variable_mean, variable_sd, int(n*1.5))
    dist.sort()
    return dist[:int(n)]

def right_tailed_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation)
    variable_sd = sd + abs(np.random.normal(0, sd_variation))
    dist = np.random.normal(variable_mean, variable_sd, int(n*1.5))
    dist.sort()
    return dist[-int(n):]

In [74]:
def rescale_to_minus_one_one(values):
    min_value = np.min(values)
    max_value = np.max(values)
    scaled_values = 2 * ((values - min_value) / (max_value - min_value)) - 1
    return scaled_values

In [75]:
def generate_samples(n, q, generator_func, *args):
    samples = generator_func(n, *args)
    samples_q = np.quantile(samples, np.linspace(0, 1, q))
    return rescale_to_minus_one_one(samples_q)

def generate_normal_samples(n=10000, q=10, mean_value=0, sd_value=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, normal_generator, mean_value, sd_value, mean_variation, sd_variation)

def generate_uniform_samples(n=10000, q=10, min_value=-1, max_value=1, min_variation=0.5, max_variation=0.5):
    return generate_samples(n, q, uniform_generator, min_value, max_value, min_variation, max_variation)

def generate_right_tailed_samples(n=10000, q=10, mean=0, sd=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, right_tailed_generator, mean, sd, mean_variation, sd_variation)

def generate_left_tailed_samples(n=10000, q=10, mean=0, sd=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, left_tailed_generator, mean, sd, mean_variation, sd_variation)

In [22]:
# def generate_multimodal_samples(n=10000, q=10, mean=0, sd=1, mean_variation=0.5, sd_variation=0.5):
#     # samples = generate_samples(n, q, left_tailed_generator, mean, sd, mean_variation, sd_variation)
#     return generate_samples(n, q, multimodal_generator, mean, sd, mean_variation, sd_variation)

In [76]:
def generate_training_data(n_samples=10000):
    data_list = {}

    # Generate samples for each distribution type
    data_list['normal'] = [generate_normal_samples() for _ in range(n_samples)]
   # data_list['bimodal'] = [generate_multimodal_samples() for _ in range(n_samples)]
    data_list['uniform'] = [generate_uniform_samples() for _ in range(n_samples)]
    data_list['right_tailed'] = [generate_right_tailed_samples() for _ in range(n_samples)]
    data_list['left_tailed'] = [generate_left_tailed_samples() for _ in range(n_samples)]

    # Combine all data into a single DataFrame
    combined_data = pd.DataFrame()
    for name, samples_list in data_list.items():
        df = pd.DataFrame(samples_list)
        df['label'] = name
        combined_data = pd.concat([combined_data, df], ignore_index=True)

    # Assign column names
    combined_data.columns = [f'V{i+1}' for i in range(combined_data.shape[1] - 1)] + ['label']

    return combined_data

## Train random forest classifier

In [77]:
def left_tailed_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation)
    variable_sd = sd + abs(np.random.normal(0, sd_variation))
    dist = np.random.normal(variable_mean, variable_sd, int(n*1.5))
    dist.sort()
    return dist[-int(n):]

In [25]:
# # data = np.quantile(multimodal_generator(100, 1,  1,1,1),np.linspace(0, 1, 10))
# # data = multimodal_generator(100, 1,  1,1,1)

# # Create a histogram plot
# plt.figure(figsize=(8, 6))
# plt.hist(data, bins=30, color='blue', alpha=0.7)
# plt.title('Histogram of Random Data')
# plt.xlabel('Values')
# plt.ylabel('Frequency')
# plt.grid(True)
# plt.show()

In [26]:
# # Quantile plot
# plt.figure(figsize=(8, 4))
# percentiles = [10,20,30,40,50,60,70,80,90,100]
# plt.plot(percentiles, np.quantile(multimodal_generator(100, 1,  1,1,1),np.linspace(0, 1, 10)), marker='o')
# plt.title('Quantile Plot')
# plt.xlabel('Percentiles')
# plt.ylabel('Quantile Values')
# plt.grid(True)
# plt.show()

In [78]:
dist_df['imp_c_quantile_scaled'] = [rescale_to_minus_one_one(dist) for dist in dist_df["imp_c_quantile"]]

In [28]:
# dist_df.columns

In [29]:
# y_true = label_encoder.inverse_transform(y_encoded).tolist()

In [30]:
# training_data = generate_training_data()

In [79]:
# Save for later; cross validation random forest for truth data

# Separate features and labels
X = pd.DataFrame(dist_df['imp_c_quantile_scaled'].tolist(), columns=[f'col{i}' for i in range(1, 11)])
y = dist_df['labeled_dist']
# Encode labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_pred_list = []
for i in range(len(X)):


  # Initialize the Random Forest Classifier
  rf_model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=50, min_samples_leaf=2)

  # Train the model
  rf_model.fit(np.array(X.drop(index= i)), np.concatenate([y[:i], y[i+1:]]))

  # Print the trained model
  print(rf_model)

  # Predict the responses for test dataset
  y_pred =  rf_model.predict(X)[i]
  y_pred_list.append(y_pred)

RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500)


...

RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


KeyboardInterrupt: 

In [80]:
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=50, min_samples_leaf=2)

# Train the model
# rf_model.fit(np.array(training_data.drop('label', axis=1)),training_data['label'])
rf_model.fit(np.array(X),y)

RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500)

In [81]:
# y_pred_list = rf_model.predict(X)
y_true = y

In [82]:
# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_list))

# Accuracy
print("Accuracy:", accuracy_score(y_true, y_pred_list))

# Precision, Recall, F1-Score
print("Classification Report:")
print(classification_report(y_true, y_pred_list))

# If it's a binary classification, calculate ROC-AUC
if len(np.unique(y_pred_list)) == 2:
    y_probs = rf_model.predict_proba(X_test)[:, 1]  # probability estimates for the positive class
    fpr, tpr, thresholds = roc_curve(y_pred_list, y_probs)
    print("AUC:", auc(fpr, tpr))


Confusion Matrix:


ValueError: Found input variables with inconsistent numbers of samples: [77, 11]

# Cross validation loop

In [90]:
# Cross-validation: loop through all dist_ids, one at a time
# subset data to exclude unlabeled distributions 1st

data = numeric_df.dropna(subset=['dist_id'])

unique_dist_ids = data['dist_id'].unique()
np.random.shuffle(unique_dist_ids)


In [36]:
# set_lists = [unique_dist_ids[i:i + 5] for i in range(0, len(unique_dist_ids), 5)]

In [37]:
# # set_lists
set_lists =[np.array([73,  1, 60, 18, 58]),
            np.array([23, 76, 51, 63, 66]),
            np.array([31, 47, 30, 20,  5]),
            np.array([15, 77, 43, 53, 12]),
            np.array([ 3, 56, 29, 75, 13]),
            np.array([70, 42, 65, 52, 54]),
            np.array([46, 41, 50, 40, 82]),
            np.array([67, 24, 84, 61,  6]),
            np.array([26, 74, 17,  2, 44]),
            np.array([81, 71, 49, 45, 88]),
            np.array([22, 11, 86, 83, 14]),
            np.array([33, 27, 25, 59, 38]),
            np.array([68, 39, 78, 69, 87]),
            np.array([16, 19, 48, 21, 79]),
            np.array([57, 72, 80, 64, 28]),
            np.array([62, 85])]

In [38]:
set_lists[5:]

[array([70, 42, 65, 52, 54]),
 array([46, 41, 50, 40, 82]),
 array([67, 24, 84, 61,  6]),
 array([26, 74, 17,  2, 44]),
 array([81, 71, 49, 45, 88]),
 array([22, 11, 86, 83, 14]),
 array([33, 27, 25, 59, 38]),
 array([68, 39, 78, 69, 87]),
 array([16, 19, 48, 21, 79]),
 array([57, 72, 80, 64, 28]),
 array([62, 85])]

In [39]:
[i for i in range(len(set_lists))][14:]

[14, 15]

In [88]:
# data

In [89]:
for set_list_id in [i for i in range(len(set_lists))][14:]:
  unique_dist_ids = set_lists[set_list_id]
  result_list = []
  import gc
  gc.collect()
  # Loop over each dist_id, leaving one out at time
  for xval_id in unique_dist_ids:
      r2_list = []
      loss_list = []
      prediction_list = []
      matched_data_list = []
      model_list = []
      # Reserve data for validation
      xval_data = data[data['dist_id'] == xval_id]

      # Matched data for training
      train_data = data[data['dist_id'] != xval_id]

      reserve_data = train_data.groupby('unique_id').sample(n=1)

      for i in range(10):
          matched_data = train_data.groupby('unique_id').sample(n=1)
          matched_data = matched_data.reset_index(drop=True)

          model, test_loss, r_squared, predictions, y_test = train_model_a(matched_data)

          predictions = predict_on_reserve(reserve_data, model, min_c, max_c)

          matched_data_list.append(matched_data)
          prediction_list.append(predictions)
          model_list.append(model)
          loss_list.append(test_loss)
          r2_list.append(r_squared)

      combined_array = np.array([predictions['predictions'].values for predictions in prediction_list])
      combined_array[combined_array < 0 ] = 0

      arr_min = np.min(combined_array.flatten())
      arr_max = np.max(combined_array.flatten())

      dist_array = [np.sort(np.array([i[j] for  i in combined_array])) for j in range(combined_array.shape[1])]
      norm_dist_array =  [np.sort((np.array([i[j] for  i in combined_array]) - arr_min) / (arr_max-arr_min)) for j in range(combined_array.shape[1])]

      keep_cols = ['water', 'trees','grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                  'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI','aet', 'def',
                  'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn','tmmx', 'vap', 'vpd', 'vs',
                  'agbd_m', 'agbd_sd', 'agbd_n']

      scaled_numeric_df = prediction_list[0][keep_cols]

      scaled_numeric_df['norm_dist_array'] = norm_dist_array
      scaled_numeric_df.dropna(inplace=True)

      scaled_numeric_df['norm_dist_array'] = scaled_numeric_df['norm_dist_array'].to_list()

      norm_dist_array = np.array([i for i in scaled_numeric_df['norm_dist_array']])

      X = scaled_numeric_df[keep_cols]
      y = norm_dist_array

      # X = scaled_numeric_df.iloc[:, :-1]
      # y = scaled_numeric_df.iloc[:, -1]

      # x = np.array(X)
      # y = np.array(norm_dist_array)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

      model = build_model(X_train.shape[1],output_shape=10)

      model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error',metrics = ['mae'])
      model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=0)
      model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error',metrics = ['mae'])
      model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=0)

      test_loss = model.evaluate(X_test, y_test)


      predictions = model.predict(xval_data[keep_cols])
      imp_c_scaled = np.sort(xval_data.groupby('unique_id').sample(n=1)['imp_c_scaled']*max_c+min_c)

      imp_c_quantiles = np.quantile(imp_c_scaled, np.linspace(0, 1,10))

      scaled_predictions = predictions*max_c+min_c
      prediction_quantiles = [np.quantile(prediction, np.linspace(0, 1,10)) for prediction in scaled_predictions]

      prediction_rf_ready_predictions = [rescale_to_minus_one_one(sorted(prediction)) for prediction in  predictions]

      # Use the trained model to make predictions
      predicted_labels_encoded = rf_model.predict(prediction_rf_ready_predictions)

      # Decode the encoded labels back to original labels
      predicted_predicted_labels = predicted_labels_encoded #here

      real_rf_ready = [rescale_to_minus_one_one(sorted(prediction)+ np.random.uniform(-0.001, 0.001, prediction.shape[0])) for prediction in  [imp_c_quantiles]]

      # Use the trained model to make predictions
      real_labels_encoded = rf_model.predict(real_rf_ready)


      # Decode the encoded labels back to original labels
      real_predicted_labels = real_labels_encoded #here

      lat_lon = [Point(xy) for xy in zip(xval_data['latitude'],xval_data['longitude'])]
      xy_sample = [Point(xy) for xy in zip(xval_data['x_sample'],xval_data['y_sample'])]
      xy_driver = [Point(xy) for xy in zip(xval_data['x_driver'],xval_data['y_driver'])]

      result_dict ={
                    "xval_id": xval_id,
                    "lat_lon": lat_lon,
                    "xy_sample": xy_sample,
                    "xy_driver": xy_driver,
                    "imp_c_scaled": imp_c_scaled,
                    "imp_c_quantiles": imp_c_quantiles,
                    "real_predicted_labels": real_predicted_labels,
                    "scaled_predictions": scaled_predictions,
                    "prediction_quantiles": prediction_quantiles,
                    "predicted_predicted_labels": predicted_predicted_labels
                  }
      result_list.append(result_dict)

  import pickle
  # Specify the filename
  filename = f'/content/drive/MyDrive/hawaii_soils/result_list_{set_list_id}.pkl'

  # Open the file in binary write mode and save the result_list
  with open(filename, 'wb') as file:
      pickle.dump(result_list, file)

  print(f"Data has been pickled and saved to {filename}.")

NameError: name 'set_lists' is not defined

In [ ]:
len(result_list)

In [ ]:
result_list = [pickle.load(open(f'/content/drive/MyDrive/hawaii_soils/result_list_{set_list_id}.pkl', 'rb')) for set_list_id in range(len(set_lists))]

In [83]:
xval_id = 0

In [92]:
r2_list = []
loss_list = []
prediction_list = []
matched_data_list = []
model_list = []
# Reserve data for validation
# xval_data = data[data['dist_id'] == xval_id]

# Matched data for training
train_data = data

reserve_data = train_data.groupby('unique_id').sample(n=1)

for i in range(10):
    matched_data = train_data.groupby('unique_id').sample(n=1)
    matched_data = matched_data.reset_index(drop=True)

    model, test_loss, r_squared, predictions, y_test = train_model_a(matched_data)

    predictions = predict_on_reserve(reserve_data, model, min_c, max_c)

    matched_data_list.append(matched_data)
    prediction_list.append(predictions)
    model_list.append(model)
    loss_list.append(test_loss)
    r2_list.append(r_squared)

combined_array = np.array([predictions['predictions'].values for predictions in prediction_list])
combined_array[combined_array < 0 ] = 0

arr_min = np.min(combined_array.flatten())
arr_max = np.max(combined_array.flatten())

dist_array = [np.sort(np.array([i[j] for  i in combined_array])) for j in range(combined_array.shape[1])]
norm_dist_array =  [np.sort((np.array([i[j] for  i in combined_array]) - arr_min) / (arr_max-arr_min)) for j in range(combined_array.shape[1])]

keep_cols = ['water', 'trees','grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
            'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI','aet', 'def',
            'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn','tmmx', 'vap', 'vpd', 'vs',
            'agbd_m', 'agbd_sd', 'agbd_n']

scaled_numeric_df = prediction_list[0][keep_cols]

scaled_numeric_df['norm_dist_array'] = norm_dist_array
scaled_numeric_df.dropna(inplace=True)

scaled_numeric_df['norm_dist_array'] = scaled_numeric_df['norm_dist_array'].to_list()

norm_dist_array = np.array([i for i in scaled_numeric_df['norm_dist_array']])

X = scaled_numeric_df[keep_cols]
y = norm_dist_array

# X = scaled_numeric_df.iloc[:, :-1]
# y = scaled_numeric_df.iloc[:, -1]

# x = np.array(X)
# y = np.array(norm_dist_array)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = build_model(X_train.shape[1],output_shape=10)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error',metrics = ['mae'])
model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=0)
model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error',metrics = ['mae'])
model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=0)

test_loss = model.evaluate(X_test, y_test)


# predictions = model.predict(xval_data[keep_cols])
# imp_c_scaled = np.sort(xval_data.groupby('unique_id').sample(n=1)['imp_c_scaled']*max_c+min_c)

# imp_c_quantiles = np.quantile(imp_c_scaled, np.linspace(0, 1,10))

# scaled_predictions = predictions*max_c+min_c
# prediction_quantiles = [np.quantile(prediction, np.linspace(0, 1,10)) for prediction in scaled_predictions]

# prediction_rf_ready_predictions = [rescale_to_minus_one_one(sorted(prediction)) for prediction in  predictions]

# # Use the trained model to make predictions
# predicted_labels_encoded = rf_model.predict(prediction_rf_ready_predictions)

# # Decode the encoded labels back to original labels
# predicted_predicted_labels = predicted_labels_encoded #here

# real_rf_ready = [rescale_to_minus_one_one(sorted(prediction)+ np.random.uniform(-0.001, 0.001, prediction.shape[0])) for prediction in  [imp_c_quantiles]]

# # Use the trained model to make predictions
# real_labels_encoded = rf_model.predict(real_rf_ready)


# # Decode the encoded labels back to original labels
# real_predicted_labels = real_labels_encoded #here

# lat_lon = [Point(xy) for xy in zip(xval_data['latitude'],xval_data['longitude'])]
# xy_sample = [Point(xy) for xy in zip(xval_data['x_sample'],xval_data['y_sample'])]
# xy_driver = [Point(xy) for xy in zip(xval_data['x_driver'],xval_data['y_driver'])]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.0207
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0178
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


...

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0171
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0157
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


<ipython-input-92-82c6ef15b89b>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_numeric_df['norm_dist_array'] = norm_dist_array
<ipython-input-92-82c6ef15b89b>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_numeric_df.dropna(inplace=True)
<ipython-input-92-82c6ef15b89b>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0015 - mae: 0.0280


In [93]:
numeric_gpd = global_scaler.transform(drivers_gpd[keep_cols])

In [94]:
pred_drivers = model.predict(numeric_gpd)

11224/11224 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step


In [95]:
predictions = pred_drivers

In [ ]:
# numeric_gpd[80]

In [96]:
scaled_predictions = predictions*max_c+min_c
prediction_quantiles = [np.quantile(prediction+ np.random.uniform(-0.001, 0.001, prediction.shape[0]), np.linspace(0, 1,10)) for prediction in predictions]

prediction_rf_ready_predictions = [rescale_to_minus_one_one(sorted(prediction)+ np.random.uniform(-0.001, 0.001, prediction.shape[0])) for prediction in  predictions]

nan_index = [np.isnan(array).any() for array in prediction_rf_ready_predictions]

prediction_rf_ready_predictions = [np.nan_to_num(array, nan=0.0) for array in prediction_rf_ready_predictions]
# # Use the trained model to make predictions
predicted_labels_encoded = rf_model.predict(prediction_rf_ready_predictions)

In [97]:
drivers_gpd['distribution_type'] = predicted_labels_encoded

In [98]:
drivers_gpd['nan_index'] = nan_index

In [99]:
quantile_df = pd.DataFrame(scaled_predictions)
quantile_df.columns = [f'Quantile_{i+1}' for i in range(quantile_df.shape[1])]


In [100]:
drivers_gpd.join(quantile_df).to_file("predicted_results.gpkg")